In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, f_classif

In [2]:
df = pd.read_csv('heart.csv')


In [3]:
imputer = SimpleImputer(strategy='median')
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

In [4]:
Q1 = df_imputed.quantile(0.25)
Q3 = df_imputed.quantile(0.75)
IQR = Q3 - Q1
df_cleaned = df_imputed[~((df_imputed < (Q1 - 1.5 * IQR)) | (df_imputed > (Q3 + 1.5 * IQR))).any(axis=1)]

In [5]:
df.shape

(1025, 14)

In [6]:
df.head

<bound method NDFrame.head of       age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak   
0      52    1   0       125   212    0        1      168      0      1.0  \
1      53    1   0       140   203    1        0      155      1      3.1   
2      70    1   0       145   174    0        1      125      1      2.6   
3      61    1   0       148   203    0        1      161      0      0.0   
4      62    0   0       138   294    1        1      106      0      1.9   
...   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
1020   59    1   1       140   221    0        1      164      1      0.0   
1021   60    1   0       125   258    0        0      141      1      2.8   
1022   47    1   0       110   275    0        0      118      1      1.0   
1023   50    0   0       110   254    0        0      159      0      0.0   
1024   54    1   0       120   188    0        1      113      0      1.4   

      slope  ca  thal  target  
0         2  

In [7]:

from sklearn.preprocessing import StandardScaler


df_cleaned = df_cleaned.copy()

scaler = StandardScaler()
df_cleaned.loc[:, df_cleaned.columns[:-1]] = scaler.fit_transform(df_cleaned.loc[:, df_cleaned.columns[:-1]])

print(df_cleaned.head())



        age       sex        cp  trestbps      chol  fbs   restecg   thalach   
0 -0.155300  0.667449 -0.907835 -0.240805 -0.675971  0.0  0.870574  0.768418  \
2  1.797168  0.667449 -0.907835  1.066034 -1.530053  0.0  0.870574 -1.138183   
3  0.820934  0.667449 -0.907835  1.262060 -0.878254  0.0  0.870574  0.458041   
5  0.495523 -1.498241 -0.907835 -1.874354  0.133160  0.0 -1.069923 -1.271202   
7  0.170111  0.667449 -0.907835  2.046163  1.054670  0.0 -1.069923 -0.251392   

      exang   oldpeak     slope        ca      thal  target  
0 -0.691987  0.026266  0.952814  2.134054  1.175946     0.0  
2  1.445114  1.550693 -2.439380 -0.702744  1.175946     0.0  
3 -0.691987 -0.926501  0.952814  0.715655  1.175946     0.0  
5 -0.691987  0.026266 -0.743283 -0.702744 -0.583397     1.0  
7  1.445114 -0.164287 -0.743283  0.715655  1.175946     0.0  


In [8]:
X = df_cleaned.drop(columns=['target'])
y = df_cleaned['target']

In [9]:
k_best = SelectKBest(score_func=f_classif, k=8)  # Select top 8 features
X_selected = k_best.fit_transform(X, y)

c:\Users\shama\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [5] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\shama\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [10]:
selected_features = k_best.get_support(indices=True)
print("Selected feature indices:", selected_features)

Selected feature indices: [ 1  2  7  8  9 10 11 12]


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)


In [12]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [13]:
y_pred = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test, y_pred))

Model Accuracy: 100.00%
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        64
         1.0       1.00      1.00      1.00        90

    accuracy                           1.00       154
   macro avg       1.00      1.00      1.00       154
weighted avg       1.00      1.00      1.00       154



In [14]:

from sklearn.svm import SVC

svm_model = SVC(kernel='rbf', random_state=42)
svm_model.fit(X_train, y_train)



y_pred = svm_model.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test, y_pred))




Model Accuracy: 85.71%
Classification Report:
               precision    recall  f1-score   support

         0.0       0.86      0.78      0.82        64
         1.0       0.85      0.91      0.88        90

    accuracy                           0.86       154
   macro avg       0.86      0.85      0.85       154
weighted avg       0.86      0.86      0.86       154



In [15]:

from sklearn.tree import DecisionTreeClassifier


dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)


y_pred = dt_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test, y_pred))



Model Accuracy: 100.00%
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        64
         1.0       1.00      1.00      1.00        90

    accuracy                           1.00       154
   macro avg       1.00      1.00      1.00       154
weighted avg       1.00      1.00      1.00       154



In [16]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report



########1. Random Forest Model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)

######## 2. Support Vector Machine Model
svm_model = SVC(kernel='rbf', random_state=42)
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)

####### 3. Decision Tree Model
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)
y_pred_dt = dt_model.predict(X_test)
accuracy_dt = accuracy_score(y_test, y_pred_dt)

####### 4. Comparison Report
print("Model Comparison Report")
print("------------------------")
print(f"Random Forest Accuracy: {accuracy_rf * 100:.2f}%")
print("Random Forest Classification Report:\n", classification_report(y_test, y_pred_rf))

print(f"Support Vector Machine Accuracy: {accuracy_svm * 100:.2f}%")
print("SVM Classification Report:\n", classification_report(y_test, y_pred_svm))

print(f"Decision Tree Accuracy: {accuracy_dt * 100:.2f}%")
print("Decision Tree Classification Report:\n", classification_report(y_test, y_pred_dt))


Model Comparison Report
------------------------
Random Forest Accuracy: 100.00%
Random Forest Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        64
         1.0       1.00      1.00      1.00        90

    accuracy                           1.00       154
   macro avg       1.00      1.00      1.00       154
weighted avg       1.00      1.00      1.00       154

Support Vector Machine Accuracy: 85.71%
SVM Classification Report:
               precision    recall  f1-score   support

         0.0       0.86      0.78      0.82        64
         1.0       0.85      0.91      0.88        90

    accuracy                           0.86       154
   macro avg       0.86      0.85      0.85       154
weighted avg       0.86      0.86      0.86       154

Decision Tree Accuracy: 100.00%
Decision Tree Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00 